In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import pickle
import lightgbm as lgb
from collections import OrderedDict, defaultdict
from tqdm import tqdm
import gc
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import sys
sys.path.append('../input/riiid-ym-train/')

In [ ]:
import psutil
import os
import time
import sys
import math
from contextlib import contextmanager

@contextmanager
def trace(title):
    t0 = time.time()
    p = psutil.Process(os.getpid())
    m0 = p.memory_info()[0] / 2. ** 30
    yield
    m1 = p.memory_info()[0] / 2. ** 30
    delta = m1 - m0
    sign = '+' if delta >= 0 else '-'
    delta = math.fabs(delta)
    print(f"[{m1:.1f}GB({sign}{delta:.1f}GB):{time.time() - t0:.1f}sec] {title} ", file=sys.stderr)

In [ ]:
EX = 61
#EX_CAT = 54

In [ ]:
with trace("riiid_tag_feature"):
    with open(f'../input/riiid-ym/EX{EX}_baseline/model.pkl','rb') as file:
        clf = pickle.load(file)
    
#with open(f'../input/riiid-ym/EX{EX_CAT}_baseline/model.pkl','rb') as file:
#    clf_cat = pickle.load(file)

In [ ]:
content_df_new = pd.read_csv('../input/riiid-ym-train/content_df_new.csv')

In [ ]:
ealsed_fill = 25440.594220232804

In [ ]:
from RiiidUtil.RiiidStateFeature import RiiidStateFeature
from RiiidUtil.RiiidStateSharedFeature import RiiidStateSharedFeature
from RiiidUtil.RiiidStateTagFeature import RiiidStateTagFeature
from RiiidUtil.RiiidStateSub import *

In [ ]:
loop_type = True

user_target_count = RiiidStateCount(['user_id'], 'answered_correctly', loop_type)
user_target_sum = RiiidStateSum(['user_id'], 'answered_correctly', loop_type)

user_part_target_count = RiiidStateCount2SourceSeries(['user_id','part'], 'answered_correctly', loop_type)
user_part_target_sum = RiiidStateSum2SourceSeries(['user_id','part'], 'answered_correctly', loop_type)

user_prior_time_currentsum = RiiidStateCurrentSum(['user_id'], 'prior_question_elapsed_time', loop_type)
user_prior_hadexp_currentsum = RiiidStateCurrentSum(['user_id'], 'prior_question_had_explanation', loop_type)

user_content_id_first = RiiidStateFirst(['user_id'], 'content_id')
user_content_id_last = RiiidStateLast(['user_id'], 'content_id')

user_timestamp_last = RiiidStateLast(['user_id'], 'timestamp')
user_timestamp_last2 = RiiidStateLag(['user_id'], 'timestamp', 2)
user_timestamp_last3 = RiiidStateLag(['user_id'], 'timestamp', 3)
user_timestamp_last10 = RiiidStateLag(['user_id'], 'timestamp', 10)

user_answered_incorrectly_sum = RiiidStateLagSum(['user_id'], 'answered_correctly', 10)
user_answered_correctly_avg_c_sum = RiiidStateSum(['user_id'], 'answered_correctly_avg_c', loop_type)
user_answered_correctly_last = RiiidStateLast(['user_id'], 'answered_correctly')
user_answered_correctly_last2 = RiiidStateLag(['user_id'], 'answered_correctly', 2)
user_answered_correctly_last3 = RiiidStateLag(['user_id'], 'answered_correctly', 3)


user_content_id_count_currentsum = RiiidStateCurrentSum(['user_id'], 'content_id_count', loop_type)
user_content_id_ptp_sum = RiiidStateSum(['user_id'], 'content_id_ptp', loop_type)

user_part_answered_correctly_timestamp = RiiidStateLast2SourceSeries(['user_id','part'], 'timestamp', loop_type)

user_cluster_answered_correctly_sum = RiiidStateSum2SourceSeries(['user_id','cluster'],'answered_correctly', loop_type)
user_cluster_answered_correctly_count = RiiidStateCount2SourceSeries(['user_id','cluster'],'answered_correctly', loop_type)

In [ ]:
riiid_state_feature = RiiidStateFeature('../input/riiid-ym-train')

In [ ]:
riiid_state_feature.append(user_target_count)
riiid_state_feature.append(user_target_sum)

riiid_state_feature.append(user_part_target_count)
riiid_state_feature.append(user_part_target_sum)

#riiid_state_feature.append(user_content_target_count)
#riiid_state_feature.append(user_content_target_sum)

riiid_state_feature.append(user_prior_time_currentsum)
riiid_state_feature.append(user_prior_hadexp_currentsum)

riiid_state_feature.append(user_content_id_first)
riiid_state_feature.append(user_content_id_last)
#riiid_state_feature.append(user_content_id_last2)

riiid_state_feature.append(user_timestamp_last)
riiid_state_feature.append(user_timestamp_last2)
riiid_state_feature.append(user_timestamp_last3)
riiid_state_feature.append(user_timestamp_last10)

riiid_state_feature.append(user_answered_incorrectly_sum)
riiid_state_feature.append(user_answered_correctly_avg_c_sum)
riiid_state_feature.append(user_answered_correctly_last)
riiid_state_feature.append(user_answered_correctly_last2)
riiid_state_feature.append(user_answered_correctly_last3)

#riiid_state_feature.append(user_tags1_answered_correctly_sum)
#riiid_state_feature.append(user_tags1_answered_correctly_count)
riiid_state_feature.append(user_content_id_count_currentsum)
riiid_state_feature.append(user_content_id_ptp_sum)
#riiid_state_feature.append(user_content_id_answered_correctly_timestamp)
#riiid_state_feature.append(user_content_id_answered_correctly_last)
riiid_state_feature.append(user_part_answered_correctly_timestamp)
#riiid_state_feature.append(user_part_answered_correctly_last)
#riiid_state_feature.append(user_tags1_timestamp_last)

riiid_state_feature.append(user_cluster_answered_correctly_sum)
riiid_state_feature.append(user_cluster_answered_correctly_count)

In [ ]:
#user_target_count.load('../input/riiid-ym-train')
with trace("riiid_state_feature.load()"):
    riiid_state_feature.load()

In [ ]:
with trace("riiid_state_feature.load()"):
    user_content_state_feature = RiiidStateSharedFeature(['user_id','content_id'], 
                                                         ['answered_correctly','answered_correctly','timestamp'], 
                                                         ['count','sum','last'], 
                                                         [loop_type,loop_type,loop_type], '../input/riiid-ym-train')

    user_content_state_feature.load()

In [ ]:
with trace("riiid_tag_feature"):
    riiid_tag_feature = RiiidStateTagFeature('../input/riiid-ym-train')
    riiid_tag_feature.load()

In [ ]:
train_columns = clf.feature_name().copy()
target = 'answered_correctly'

In [ ]:
import riiideducation
env = riiideducation.make_env()
iter_test = env.iter_test()
set_predict = env.predict

In [ ]:
previous_test_df = None
for (test_df, sample_prediction_df) in iter_test:
    try:
        if previous_test_df is not None:
            previous_test_df[target] = eval(test_df["prior_group_answers_correct"].iloc[0])
            previous_test_df = previous_test_df[previous_test_df['content_type_id']==0].reset_index(drop=True)
            #previous_test_df = pd.merge(previous_test_df, content_df, on='content_id',  how="left")
            #previous_test_df = pd.merge(previous_test_df, questions , left_on='content_id', right_on='question_id', how='left')
            previous_test_df = pd.concat([previous_test_df, content_df_new.reindex(previous_test_df['content_id'].values).reset_index(drop=True)], axis=1)
            previous_test_df['prior_question_had_explanation'] = previous_test_df['prior_question_had_explanation'].fillna(False).astype('int8')
            previous_test_df['prior_question_elapsed_time'] = previous_test_df['prior_question_elapsed_time'].fillna(ealsed_fill)
            
            #user_target_count.make_run(previous_test_df, True)
            #with trace("riiid_state_feature.only_update(previous_test_df)"):
            riiid_state_feature.only_update(previous_test_df)
            #previous_test_df = user_content_state_feature.run(previous_test_df, True)
            #with trace("user_content_state_feature.total_loop_only_update(previous_test_df)"):
            user_content_state_feature.total_loop_only_update(previous_test_df)
                
            riiid_tag_feature.total_loop_only_update(previous_test_df)
    
        previous_test_df = test_df.copy()
        test_df = test_df[test_df['content_type_id'] == 0].reset_index(drop=True)
        #test_df = pd.merge(test_df, content_df, on='content_id',  how="left")
        #test_df = pd.merge(test_df,questions , left_on='content_id', right_on='question_id', how='left')
        test_df = pd.concat([test_df, content_df_new.reindex(test_df['content_id'].values).reset_index(drop=True)], axis=1)
        test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].fillna(False).astype('int8')
        test_df['prior_question_elapsed_time'] = test_df['prior_question_elapsed_time'].fillna(ealsed_fill)
        #test_df['content_time_mean'] = test_df['content_id'].map(content_time_mean_dict)
        
        test_df[target] = np.nan
        #user_target_count.make_run(test_df, False)
        #with trace("riiid_state_feature.run(test_df, False)"):
        test_df = riiid_state_feature.run(test_df, False)
        #test_df = user_content_state_feature.run(test_df, False)
        #with trace("user_content_state_feature.total_loop_inference(test_df)"):
        test_df = user_content_state_feature.total_loop_inference(test_df)
        test_df = riiid_tag_feature.total_loop_inference(test_df)
        
        test_df['user_id_answered_correctly_mean'] = test_df['user_id_answered_correctly_sum']/test_df['user_id_answered_correctly_count']
        test_df['user_id_content_id_answered_correctly_mean'] = test_df['user_id_content_id_answered_correctly_sum']/test_df['user_id_content_id_answered_correctly_count']
        test_df['user_id_part_answered_correctly_mean'] = test_df['user_id_part_answered_correctly_sum']/test_df['user_id_part_answered_correctly_count']
        test_df['upm_div_cm'] = test_df['user_id_part_answered_correctly_mean']/test_df['answered_correctly_avg_c']
        #test_df.loc[np.isinf(test_df['upm_div_cm']),'upm_div_cm'] = 1
        test_df['elapsed_time_mean'] = test_df['user_id_prior_question_elapsed_time_currentsum']/(test_df['user_id_answered_correctly_count']+1)
        test_df['ptime_div_em'] = test_df['prior_question_elapsed_time']/test_df['elapsed_time_mean']
        test_df['same_content'] = test_df['content_id'] == test_df['user_id_content_id_last']
        test_df['same_content'] = test_df['same_content'].astype('int8')
        test_df['timediff'] = test_df['timestamp'] - test_df['user_id_timestamp_last']
        test_df['timediff2'] = test_df['timestamp'] - test_df['user_id_timestamp_last2']
        test_df['timediff3'] = test_df['timestamp'] - test_df['user_id_timestamp_last3']
        test_df['timediff10'] = test_df['timestamp'] - test_df['user_id_timestamp_last10']
        test_df.loc[test_df['timediff']==0,'timediff'] = np.nan
        test_df['timediff'] = test_df['timediff'].fillna(method='ffill')
        test_df.loc[test_df['user_id_answered_correctly_count']==0,'timediff'] = np.nan
        test_df.loc[test_df['timediff2']==0,'timediff2'] = np.nan
        test_df['lag_diff_elapsed'] = test_df['prior_question_elapsed_time'] - test_df['timediff']
        test_df['lag_diff_elapsed2'] = test_df['prior_question_elapsed_time'] - test_df['timediff2']
        
        
        test_df['user_id_answered_correctly_avg_c_sum_mean'] = test_df['user_id_answered_correctly_avg_c_sum']/test_df['user_id_answered_correctly_count']
        test_df['user_id_prior_question_had_explanation_mean'] = test_df['user_id_prior_question_had_explanation_currentsum']/(test_df['user_id_answered_correctly_count']+1)
        test_df['user_id_content_id_ptp_mean'] = test_df['user_id_content_id_ptp_sum']/test_df['user_id_answered_correctly_count']
        test_df['user_id_content_id_timestamp_diff'] = test_df['timestamp'] - test_df['user_id_content_id_timestamp_last']
        #test_df['user_id_content_id_user_id_answered_correctly_count_last_diff'] = test_df['user_id_answered_correctly_count'] - test_df['user_id_content_id_user_id_answered_correctly_count_last']
        test_df['user_id_part_timestamp_diff'] = test_df['timestamp'] - test_df['user_id_part_timestamp_last']
        #test_df['user_id_part_user_id_answered_correctly_count_last_diff'] = test_df['user_id_answered_correctly_count'] - test_df['user_id_part_user_id_answered_correctly_count_last']
        
        test_df['answered_correctly_avg_c_div_b'] = test_df['answered_correctly_avg_c']/test_df['answered_correctly_avg_b']
        test_df['user_id_cluster_answered_correctly_mean'] = test_df['user_id_cluster_answered_correctly_sum'] / test_df['user_id_cluster_answered_correctly_count']  
        
        test_df['user_id_tags1_answer_mean'] = test_df['user_id_tags1_answer_sum']/test_df['user_id_tags1_answer_count']
        test_df['user_id_tags2_answer_mean'] = test_df['user_id_tags2_answer_sum']/test_df['user_id_tags2_answer_count']
        test_df['user_id_tags3_answer_mean'] = test_df['user_id_tags3_answer_sum']/test_df['user_id_tags3_answer_count']
        test_df['user_id_tags4_answer_mean'] = test_df['user_id_tags4_answer_sum']/test_df['user_id_tags4_answer_count']
        test_df['user_id_tags1_answer_diff'] = test_df['timestamp'] - test_df['user_id_tags1_answer_last']
        test_df['user_id_tags2_answer_diff'] = test_df['timestamp'] - test_df['user_id_tags2_answer_last']
        test_df['user_id_tags3_answer_diff'] = test_df['timestamp'] - test_df['user_id_tags3_answer_last']
        test_df['user_id_tags4_answer_diff'] = test_df['timestamp'] - test_df['user_id_tags4_answer_last']

        #lgb_pred = clf.predict(test_df[train_columns])
        #cat_pred =  clf_cat.predict_proba(test_df[train_columns])[:,1]
        test_df[target] = clf.predict(test_df[train_columns]) #lgb_pred*0.5 + cat_pred*0.5
        set_predict(test_df[['row_id', target]])
        #del lgb_pred
        #del cat_pred
        #gc.collect()
    except BaseException as e:
        print(e)
        continue

In [ ]:
test_df